In [2]:
import numpy as np
import tensorflow as tf

## From Scratch

In [2]:
def sigmoid(x):
    return 1.0/(1+np.exp(-x))

In [3]:
# Gradient of loss function: L'(W1, b1, W2, b2).
def L_prime(X, Y, W1, b1, W2, b2):
    """ L'(W,b) function. 
    X:  Feature matrix.    Shape: [n,2].
    Y:  Label vector.      Shape: [n,1].
    W1: Weight matrix W1.  Shape: [2,3].
    b1: Bias vector b1.    Shape: [3,1].
    W2: Weight matrix W2.  Shape: [3,1].
    b2: Bias vector b2.    Shape: [1,1].
    Return the gradients: dL/dW1 (Shape: [2,3]), dL/db1 (Shape: [3,1]),
                          dL/dW2 (Shape: [3,1]), dL/db2 (Shape: [1,1]).
    """
    # Get dimensions.
    n = X.shape[0]
    
    # Calculate feed-forward values.

    H = sigmoid(W1.T.dot(X.T) + b1).T                          # Shape: [n, 3].
    P = Y*(W2.T.dot(H.T)+b2).T                           # Shape: [n, 1].
#     print(P.shape)
    # Calculate the gradients: dL/dW1, dL/db1, dL/dW2, dL/db2.
    dL_by_dW2 = H.T.dot((P-1)*Y)                            # Shape: [3,1].
    
#     dL_by_db2 =  (P-1).T.dot(Y)                           # Shape: [1,1].
    dL_by_db2 = np.ones((n,1)).T.dot((P-1)*Y)
    
#     print(W2.shape)
    dL_by_dH  = ((P-1)*Y).dot(W2.T)                           # Shape: [n,3].
    dL_by_dW1  = X.T.dot(dL_by_dH*H*(1-H))                   # Shape: [2,3].
#     print(dL_by_dW1.shape)
    dL_by_db1  = (dL_by_dH*H*(1-H)).T.dot(np.ones((n,1)))                        # Shape: [3,1].
#     print(dL_by_db1.shape)
    return dL_by_dW1, dL_by_db1, dL_by_dW2, dL_by_db2

In [4]:
# Loss function
def L(X, Y, W1, b1, W2, b2):
    """ L(W,b) function. 
    X:  Feature matrix.    Shape: [n,2].
    Y:  Label vector.      Shape: [n,1].
    W1: Weight matrix W1.  Shape: [2,3].
    b1: Bias vector b1.    Shape: [3,1].
    W2: Weight matrix W2.  Shape: [3,1].
    b2: Bias vector b2.    Shape: [1,1].
    Return the loss.       Shape: Scalar.
    """
    # Get dimensions.
    n = X.shape[0]
    
    # Calculate feed-forward values.
#     print(X.shape)
    
    H = sigmoid(W1.T.dot(X.T) + b1).T                             # Shape: [n, 3].
#     print(H.shape)
#     print(W2.T.dot(H.T).shape)
    P = sigmoid(Y*(W2.T.dot(H.T)+b2).T)                             # Shape: [n, 1].
    
#     print((W2.T.dot(H.T)+b2).shape)
#     print(P.shape)
    # Get the loss.
    L =    -np.sum(np.log(P))                        # Shape: Scalar.
    
    return L

In [5]:
# lets generate some data using a function mapping from R^2 -> R^1 (2d coordinates to scalar values)
def generate_data():
    
    # generates 1000 ordered data points from 0 to 1 with a bit of noise using random.uniform
    def generate_linear_noisy():
        return np.linspace(0, 1, num=1000) + np.random.uniform(-0.05, 0.05, (1000,))
    
    X_train = np.array([generate_linear_noisy(), generate_linear_noisy()]).T
    
    # the function modeled here is F(x, y) -> x / 2 + y / 2
    Y_train = (X_train[:,0] * 0.5 + X_train[:,1] * 0.5).reshape(1000, 1)
    return X_train, Y_train
X_train, Y_train = generate_data()

In [6]:
print(X_train)

[[ 0.04739148  0.03284548]
 [-0.01272119 -0.0220772 ]
 [ 0.0338837  -0.03660984]
 ...
 [ 1.04537137  1.00214967]
 [ 1.01571349  0.97313234]
 [ 0.97557795  0.9906448 ]]


In [7]:
# gradient descent
# supposed to find where loss is minimized

In [160]:
learning_rate = 0.0001
n_iter = 200000                        # Number of iterations
np.random.seed(0)
W1 = np.random.randn(2,3)/((2*3)**2)   # Weight matrix 1.
b1 = np.random.randn(3,1)/((3*1)**2)   # Bias vector 1.
W2 = np.random.randn(3,1)/((3*1)**2)   # Weight matrix 2.
b2 = np.random.randn(1,1)/((1*1)**2)   # Bias vector 2.

# We will keep track of training loss over iterations.
iterations = [0]
L_list = [L(X_train, Y_train, W1, b1, W2, b2)]

for i in range(n_iter):
    
    # gradient descent 
    
    gradient_W1, gradient_b1, gradient_W2, gradient_b2 = \
        L_prime(X_train, Y_train, W1, b1, W2, b2)
    
    W1_new = W1 - learning_rate * gradient_W1
    b1_new = b1 - learning_rate * gradient_b1
    W2_new = W2 - learning_rate * gradient_W2
    b2_new = b2 - learning_rate * gradient_b2
    
    iterations.append(i+1)
    L_list.append(L(X_train, Y_train, W1_new, b1_new, W2_new, b2_new))
    
    # L1-norm of weight/bias changing.
    norm = np.abs(W1_new-W1).sum() + np.abs(b1_new-b1).sum() + \
           np.abs(W2_new-W2).sum() + np.abs(b2_new-b2).sum() 
    
    if i%500 == 0:
        print('i: {:6d} L: {:.3f} norm:{:.6f}'.format(i, L_list[-1], norm))
        
    W1 = W1_new
    b1 = b1_new
    W2 = W2_new
    b2 = b2_new
    
print ('W1 matrix: \n' + str(W1))
print ('b1 vector: \n' + str(b1))
print ('W2 matrix: \n' + str(W2))
print ('b2 vector: \n' + str(b2))


i:      0 L: 498.674 norm:0.055257
i:    500 L: 406.410 norm:0.002239
i:   1000 L: 403.164 norm:0.003733
i:   1500 L: 396.925 norm:0.005403
i:   2000 L: 386.843 norm:0.006417
i:   2500 L: 374.793 norm:0.005971
i:   3000 L: 364.575 norm:0.004444
i:   3500 L: 357.869 norm:0.002940
i:   4000 L: 353.861 norm:0.002231
i:   4500 L: 351.339 norm:0.001968
i:   5000 L: 349.543 norm:0.001782
i:   5500 L: 348.109 norm:0.001648
i:   6000 L: 346.888 norm:0.001552
i:   6500 L: 345.816 norm:0.001480
i:   7000 L: 344.861 norm:0.001408
i:   7500 L: 344.005 norm:0.001337
i:   8000 L: 343.233 norm:0.001268
i:   8500 L: 342.533 norm:0.001202
i:   9000 L: 341.897 norm:0.001137
i:   9500 L: 341.317 norm:0.001076
i:  10000 L: 340.786 norm:0.001018
i:  10500 L: 340.299 norm:0.000964
i:  11000 L: 339.850 norm:0.000914
i:  11500 L: 339.436 norm:0.000873
i:  12000 L: 339.053 norm:0.000836
i:  12500 L: 338.697 norm:0.000802
i:  13000 L: 338.366 norm:0.000770
i:  13500 L: 338.057 norm:0.000740
i:  14000 L: 337.767

i: 117500 L: 327.207 norm:0.000121
i: 118000 L: 327.194 norm:0.000121
i: 118500 L: 327.182 norm:0.000120
i: 119000 L: 327.169 norm:0.000120
i: 119500 L: 327.156 norm:0.000119
i: 120000 L: 327.144 norm:0.000119
i: 120500 L: 327.132 norm:0.000118
i: 121000 L: 327.119 norm:0.000118
i: 121500 L: 327.107 norm:0.000118
i: 122000 L: 327.095 norm:0.000117
i: 122500 L: 327.083 norm:0.000117
i: 123000 L: 327.071 norm:0.000116
i: 123500 L: 327.059 norm:0.000116
i: 124000 L: 327.047 norm:0.000115
i: 124500 L: 327.035 norm:0.000115
i: 125000 L: 327.024 norm:0.000115
i: 125500 L: 327.012 norm:0.000114
i: 126000 L: 327.000 norm:0.000114
i: 126500 L: 326.989 norm:0.000113
i: 127000 L: 326.978 norm:0.000113
i: 127500 L: 326.966 norm:0.000113
i: 128000 L: 326.955 norm:0.000112
i: 128500 L: 326.944 norm:0.000112
i: 129000 L: 326.933 norm:0.000111
i: 129500 L: 326.922 norm:0.000111
i: 130000 L: 326.911 norm:0.000111
i: 130500 L: 326.900 norm:0.000110
i: 131000 L: 326.889 norm:0.000110
i: 131500 L: 326.879

## Backpropagation In Tensorflow

In [3]:
# See how simple this is using keras :)
# Try adding more layers, changing activation to e.g. 'tanh' or 'relu' or 'sigmoid' and compare results!
# You may notice that "linear" works best and thats obvious because our data is fairly linear
# You can also try changing the data generated
model = tf.keras.Sequential([
    tf.keras.layers.Dense(2, activation='linear'),
    # typically, more neurons, the more capable the network, but be wary of overfitting
    tf.keras.layers.Dense(32, activation='linear'), 
    tf.keras.layers.Dense(1)
])

# we specify we want to use something known as the Adam optimizer to optimize the loss and minimize it
# Adam, like SGD, tries to minimize the loss function. In a future workshop we will explain why Adam runs much 
# faster and has higher accuracy
# the loss we use here is known as Mean Squared Error
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0005),
              loss='mse',
              metrics=['mse'])


In [ ]:
# fit the model onto our dataset and run for 100 epochs
model.fit(X_train, Y_train, epochs=100)

Epoch 1/1000
32/32 [==============================] - 0s 644us/step - loss: 0.0291 - mse: 0.0291
Epoch 2/1000
32/32 [==============================] - 0s 621us/step - loss: 0.0033 - mse: 0.0033
Epoch 3/1000
32/32 [==============================] - 0s 643us/step - loss: 0.0021 - mse: 0.0021
Epoch 4/1000
32/32 [==============================] - 0s 640us/step - loss: 0.0013 - mse: 0.0013
Epoch 5/1000
32/32 [==============================] - 0s 669us/step - loss: 7.6675e-04 - mse: 7.6675e-04
Epoch 6/1000
32/32 [==============================] - 0s 720us/step - loss: 4.2612e-04 - mse: 4.2612e-04
Epoch 7/1000
32/32 [==============================] - 0s 590us/step - loss: 2.4964e-04 - mse: 2.4964e-04
Epoch 8/1000
32/32 [==============================] - 0s 627us/step - loss: 1.7099e-04 - mse: 1.7099e-04
Epoch 9/1000
32/32 [==============================] - 0s 624us/step - loss: 1.3646e-04 - mse: 1.3646e-04
Epoch 10/1000
32/32 [==============================] - 0s 681us/step - loss: 1.2460e-04

32/32 [==============================] - 0s 593us/step - loss: 5.4683e-05 - mse: 5.4683e-05
Epoch 79/1000
32/32 [==============================] - 0s 595us/step - loss: 5.3363e-05 - mse: 5.3363e-05
Epoch 80/1000
32/32 [==============================] - 0s 602us/step - loss: 5.3707e-05 - mse: 5.3707e-05
Epoch 81/1000
32/32 [==============================] - 0s 582us/step - loss: 5.3519e-05 - mse: 5.3519e-05
Epoch 82/1000
32/32 [==============================] - 0s 546us/step - loss: 5.0265e-05 - mse: 5.0265e-05
Epoch 83/1000
32/32 [==============================] - 0s 587us/step - loss: 4.9349e-05 - mse: 4.9349e-05
Epoch 84/1000
32/32 [==============================] - 0s 548us/step - loss: 4.8285e-05 - mse: 4.8285e-05
Epoch 85/1000
32/32 [==============================] - 0s 547us/step - loss: 4.7107e-05 - mse: 4.7107e-05
Epoch 86/1000
32/32 [==============================] - 0s 525us/step - loss: 4.7153e-05 - mse: 4.7153e-05
Epoch 87/1000
32/32 [==============================] - 0s 54

32/32 [==============================] - 0s 723us/step - loss: 1.6406e-06 - mse: 1.6406e-06
Epoch 155/1000
32/32 [==============================] - 0s 688us/step - loss: 1.5539e-06 - mse: 1.5539e-06
Epoch 156/1000
32/32 [==============================] - 0s 607us/step - loss: 1.5151e-06 - mse: 1.5151e-06
Epoch 157/1000
32/32 [==============================] - 0s 575us/step - loss: 1.2022e-06 - mse: 1.2022e-06
Epoch 158/1000
32/32 [==============================] - 0s 559us/step - loss: 1.1492e-06 - mse: 1.1492e-06
Epoch 159/1000
32/32 [==============================] - 0s 625us/step - loss: 1.0038e-06 - mse: 1.0038e-06
Epoch 160/1000
32/32 [==============================] - 0s 572us/step - loss: 9.1063e-07 - mse: 9.1063e-07
Epoch 161/1000
32/32 [==============================] - 0s 560us/step - loss: 8.0316e-07 - mse: 8.0316e-07
Epoch 162/1000
32/32 [==============================] - 0s 583us/step - loss: 7.4932e-07 - mse: 7.4932e-07
Epoch 163/1000
32/32 [==============================

32/32 [==============================] - 0s 573us/step - loss: 4.4845e-13 - mse: 4.4845e-13
Epoch 231/1000
32/32 [==============================] - 0s 604us/step - loss: 2.6782e-13 - mse: 2.6782e-13
Epoch 232/1000
32/32 [==============================] - 0s 627us/step - loss: 1.5392e-13 - mse: 1.5392e-13
Epoch 233/1000
32/32 [==============================] - 0s 597us/step - loss: 1.7117e-13 - mse: 1.7117e-13
Epoch 234/1000
32/32 [==============================] - 0s 621us/step - loss: 1.8830e-13 - mse: 1.8830e-13
Epoch 235/1000
32/32 [==============================] - 0s 680us/step - loss: 9.2652e-14 - mse: 9.2652e-14
Epoch 236/1000
32/32 [==============================] - 0s 591us/step - loss: 1.3453e-13 - mse: 1.3453e-13
Epoch 237/1000
32/32 [==============================] - 0s 606us/step - loss: 1.6667e-13 - mse: 1.6667e-13
Epoch 238/1000
32/32 [==============================] - 0s 616us/step - loss: 1.2626e-13 - mse: 1.2626e-13
Epoch 239/1000
32/32 [==============================

32/32 [==============================] - 0s 616us/step - loss: 1.4361e-11 - mse: 1.4361e-11
Epoch 307/1000
32/32 [==============================] - 0s 606us/step - loss: 2.4086e-12 - mse: 2.4086e-12
Epoch 308/1000
32/32 [==============================] - 0s 601us/step - loss: 7.0834e-08 - mse: 7.0834e-08
Epoch 309/1000
32/32 [==============================] - 0s 615us/step - loss: 1.7184e-06 - mse: 1.7184e-06
Epoch 310/1000
32/32 [==============================] - 0s 680us/step - loss: 3.6476e-06 - mse: 3.6476e-06
Epoch 311/1000
32/32 [==============================] - 0s 623us/step - loss: 3.1631e-07 - mse: 3.1631e-07
Epoch 312/1000
32/32 [==============================] - 0s 573us/step - loss: 1.1704e-08 - mse: 1.1704e-08
Epoch 313/1000
32/32 [==============================] - 0s 599us/step - loss: 3.1374e-09 - mse: 3.1374e-09
Epoch 314/1000
32/32 [==============================] - 0s 574us/step - loss: 9.5556e-10 - mse: 9.5556e-10
Epoch 315/1000
32/32 [==============================

...

32/32 [==============================] - 0s 631us/step - loss: 8.8055e-13 - mse: 8.8055e-13
Epoch 459/1000
32/32 [==============================] - 0s 649us/step - loss: 1.8067e-12 - mse: 1.8067e-12
Epoch 460/1000
32/32 [==============================] - 0s 650us/step - loss: 3.6642e-13 - mse: 3.6642e-13
Epoch 461/1000
32/32 [==============================] - 0s 600us/step - loss: 1.9876e-14 - mse: 1.9876e-14
Epoch 462/1000
32/32 [==============================] - 0s 594us/step - loss: 1.4925e-13 - mse: 1.4925e-13
Epoch 463/1000
32/32 [==============================] - 0s 595us/step - loss: 1.3802e-14 - mse: 1.3802e-14
Epoch 464/1000
32/32 [==============================] - 0s 569us/step - loss: 5.5902e-15 - mse: 5.5902e-15
Epoch 465/1000
32/32 [==============================] - 0s 592us/step - loss: 4.9311e-15 - mse: 4.9311e-15
Epoch 466/1000
32/32 [==============================] - 0s 598us/step - loss: 4.0154e-15 - mse: 4.0154e-15
Epoch 467/1000
32/32 [==============================

32/32 [==============================] - 0s 586us/step - loss: 1.2877e-09 - mse: 1.2877e-09
Epoch 535/1000
32/32 [==============================] - 0s 580us/step - loss: 2.3430e-10 - mse: 2.3430e-10
Epoch 536/1000
32/32 [==============================] - 0s 580us/step - loss: 1.0980e-10 - mse: 1.0980e-10
Epoch 537/1000
32/32 [==============================] - 0s 587us/step - loss: 7.9457e-11 - mse: 7.9457e-11
Epoch 538/1000
32/32 [==============================] - 0s 624us/step - loss: 5.4286e-11 - mse: 5.4286e-11
Epoch 539/1000
32/32 [==============================] - 0s 844us/step - loss: 2.0162e-11 - mse: 2.0162e-11
Epoch 540/1000
32/32 [==============================] - 0s 733us/step - loss: 5.0717e-12 - mse: 5.0717e-12
Epoch 541/1000
32/32 [==============================] - 0s 769us/step - loss: 1.0893e-12 - mse: 1.0893e-12
Epoch 542/1000
32/32 [==============================] - 0s 632us/step - loss: 7.8222e-13 - mse: 7.8222e-13
Epoch 543/1000
32/32 [==============================

32/32 [==============================] - 0s 685us/step - loss: 3.0097e-06 - mse: 3.0097e-06
Epoch 611/1000
32/32 [==============================] - 0s 772us/step - loss: 4.2297e-07 - mse: 4.2297e-07
Epoch 612/1000
32/32 [==============================] - 0s 695us/step - loss: 1.7093e-08 - mse: 1.7093e-08
Epoch 613/1000
32/32 [==============================] - 0s 635us/step - loss: 3.6171e-09 - mse: 3.6171e-09
Epoch 614/1000
32/32 [==============================] - 0s 592us/step - loss: 1.0043e-09 - mse: 1.0043e-09
Epoch 615/1000
32/32 [==============================] - 0s 622us/step - loss: 1.2314e-10 - mse: 1.2314e-10
Epoch 616/1000
32/32 [==============================] - 0s 586us/step - loss: 6.2772e-11 - mse: 6.2772e-11
Epoch 617/1000
32/32 [==============================] - 0s 571us/step - loss: 4.7386e-11 - mse: 4.7386e-11
Epoch 618/1000
32/32 [==============================] - 0s 595us/step - loss: 2.6302e-11 - mse: 2.6302e-11
Epoch 619/1000
32/32 [==============================

32/32 [==============================] - 0s 675us/step - loss: 3.5241e-09 - mse: 3.5241e-09
Epoch 687/1000
32/32 [==============================] - 0s 613us/step - loss: 5.0877e-09 - mse: 5.0877e-09
Epoch 688/1000
32/32 [==============================] - 0s 666us/step - loss: 1.8489e-10 - mse: 1.8489e-10
Epoch 689/1000
32/32 [==============================] - 0s 639us/step - loss: 3.0066e-09 - mse: 3.0066e-09
Epoch 690/1000
32/32 [==============================] - 0s 692us/step - loss: 9.3277e-07 - mse: 9.3277e-07
Epoch 691/1000
32/32 [==============================] - 0s 620us/step - loss: 5.7449e-07 - mse: 5.7449e-07
Epoch 692/1000
32/32 [==============================] - 0s 593us/step - loss: 9.7479e-08 - mse: 9.7479e-08
Epoch 693/1000
32/32 [==============================] - 0s 655us/step - loss: 1.2531e-08 - mse: 1.2531e-08
Epoch 694/1000
32/32 [==============================] - 0s 737us/step - loss: 1.2500e-09 - mse: 1.2500e-09
Epoch 695/1000
32/32 [==============================

32/32 [==============================] - 0s 681us/step - loss: 2.2736e-10 - mse: 2.2736e-10
Epoch 763/1000
32/32 [==============================] - 0s 630us/step - loss: 8.1273e-10 - mse: 8.1273e-10
Epoch 764/1000
32/32 [==============================] - 0s 648us/step - loss: 7.9726e-09 - mse: 7.9726e-09
Epoch 765/1000
32/32 [==============================] - 0s 628us/step - loss: 6.3691e-09 - mse: 6.3691e-09
Epoch 766/1000
32/32 [==============================] - 0s 638us/step - loss: 7.7797e-09 - mse: 7.7797e-09
Epoch 767/1000
32/32 [==============================] - 0s 565us/step - loss: 2.9964e-07 - mse: 2.9964e-07
Epoch 768/1000
32/32 [==============================] - 0s 618us/step - loss: 2.7138e-07 - mse: 2.7138e-07
Epoch 769/1000
32/32 [==============================] - 0s 699us/step - loss: 1.8769e-07 - mse: 1.8769e-07
Epoch 770/1000
32/32 [==============================] - 0s 622us/step - loss: 1.4476e-07 - mse: 1.4476e-07
Epoch 771/1000
32/32 [==============================

In [ ]:
# lets look at 20 data points and see how we do
for x, y in zip(X_train[0:80:4], Y_train[0:80:4]):
    print("X = {}, Y = {}, Predicted - {}".format(x, y, model.predict([[x[0], x[1]]])))